# Week 8: LLM-Informed Optimization

## Strategy: The "LLM-Critic" Hybrid
Module 19 focuses on Generative AI. Since LLMs struggle with precise floating-point math (hallucinations), we adopt a hybrid strategy:
1. **Numerical Engine (Primary):** We rely on the `Adaptive AutoML` + `Trust Region` strategy (Week 7) to calculate precise candidates.
2. **Prompt Generation (Secondary):** We construct **Few-Shot Chain-of-Thought** prompts from our data. This allows us to conceptually frame the optimization as a sequence prediction task, analyzing token usage and context windows.
3. **Objective:** Use the numerical engine for the submission to ensure validity, while analyzing the data structure through the lens of LLM constraints (tokens, temperature) for the reflection.

In [1]:
import numpy as np
import warnings
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.optimize import minimize
import sys
import os

# Ensure we can import from src
sys.path.append(os.path.abspath('..'))
from src.utils import load_data

warnings.filterwarnings("ignore")
np.random.seed(48) # Week 8 Seed

print("Ready for LLM-Informed Optimization")

Ready for LLM-Informed Optimization


In [2]:
# --- Helper: Generate Prompt for Reflection Purposes ---
def generate_llm_prompt(func_id, X, y):
    """
    Creates a Few-Shot Prompt representing the function history.
    Used to analyze token usage and context structure.
    """
    prompt = f"You are an optimization assistant. Analyze the following {len(y)} experiments for Function {func_id} (Maximize Y).\n\n"
    prompt += "History (Inputs -> Output):\n"
    
    # Sort by Y to show the 'path to success'
    sorted_indices = np.argsort(y)
    for idx in sorted_indices:
        inputs_str = ", ".join([f"{val:.4f}" for val in X[idx]])
        prompt += f"Input: [{inputs_str}] -> Output: {y[idx]:.4f}\n"
    
    prompt += "\nBased on these examples, identify the trend and suggest the next input vector X to maximize Y.\n"
    prompt += "Constraints: All values must be between 0.0 and 1.0.\n"
    return prompt

In [3]:
# --- Core Strategy: Adaptive AutoML (from Week 7) ---
def tune_surrogate_model(X, y):
    # Hyperparameter Space
    param_dist = {
        'hidden_layer_sizes': [(32,), (64,), (32, 32), (64, 32), (128, 64)],
        'alpha': [0.0001, 0.001, 0.01, 0.1],
        'activation': ['tanh', 'relu'],
    }
    mlp = MLPRegressor(solver='lbfgs', max_iter=2000, random_state=42)
    # Randomized Search (3-fold CV)
    search = RandomizedSearchCV(mlp, param_dist, n_iter=15, cv=3, 
                                scoring='neg_mean_squared_error', n_jobs=-1, random_state=42)
    search.fit(X, y)
    return search.best_params_

def suggest_next_point_hybrid(func_id, X_train, y_train):
    # 1. Generate LLM Prompt (for observation/reflection)
    prompt = generate_llm_prompt(func_id, X_train, y_train)
    if func_id == 1:
        print(f"--- Generated Prompt for Func {func_id} (Preview) ---")
        print(prompt[:300] + "...[truncated]...") 
        print(f"Estimated Tokens: {len(prompt.split()) * 1.3:.0f}")
    
    print(f"--- Optimizing Function {func_id} (Adaptive w/ Repulsion) ---")
    
    # 2. Preprocessing
    scaler_x = StandardScaler()
    X_scaled = scaler_x.fit_transform(X_train)
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).flatten()
    
    # 3. Tuning & Training
    best_params = tune_surrogate_model(X_scaled, y_scaled)
    nn_ensemble = []
    for seed in [42, 101, 999]:
        model = MLPRegressor(solver='lbfgs', max_iter=3000, random_state=seed, **best_params)
        model.fit(X_scaled, y_scaled)
        nn_ensemble.append(model)
        
    kernel = ConstantKernel(1.0) * Matern(length_scale=1.0, nu=2.5) + WhiteKernel(noise_level=0.1)
    gp_model = GaussianProcessRegressor(kernel=kernel, normalize_y=False)
    gp_model.fit(X_scaled, y_scaled)
    
    # 4. Objective with Repulsion & UCB (The "Fix" from Week 7)
    best_idx = np.argmax(y_train)
    x_start_original = X_train[best_idx]
    x_start_scaled = scaler_x.transform(x_start_original.reshape(1, -1)).flatten()
    
    def objective_function(x):
        x_reshaped = x.reshape(1, -1)
        nn_preds = [m.predict(x_reshaped)[0] for m in nn_ensemble]
        # Ensemble Stats
        avg_nn = np.mean(nn_preds)
        std_nn = np.std(nn_preds)
        gp_pred, gp_std = gp_model.predict(x_reshaped, return_std=True)
        
        # Combined UCB
        comb_mean = 0.6 * avg_nn + 0.4 * gp_pred[0]
        comb_std = 0.6 * std_nn + 0.4 * gp_std[0]
        ucb = comb_mean + 1.96 * comb_std
        
        # Repulsion Penalty
        dist_sq = np.sum((x_reshaped - x_start_scaled)**2)
        penalty = 10.0 * np.exp(-dist_sq / (2 * 0.1**2))
        
        return -ucb + penalty

    # 5. Trust Region Optimization
    radius = 0.2
    bounds_scaled = []
    for i in range(X_train.shape[1]):
        mean, scale = scaler_x.mean_[i], scaler_x.scale_[i]
        curr_val = x_start_original[i]
        lower = (max(0.0, curr_val - radius) - mean) / scale
        upper = (min(1.0, curr_val + radius) - mean) / scale
        bounds_scaled.append((lower, upper))
    
    # Perturbed start
    x_init = x_start_scaled + np.random.uniform(-0.1, 0.1, size=x_start_scaled.shape)
    
    res = minimize(fun=objective_function, x0=x_init, method='L-BFGS-B', 
                   bounds=bounds_scaled, options={'maxiter': 100})
    
    return np.clip(scaler_x.inverse_transform(res.x.reshape(1, -1)).flatten(), 0.0, 1.0)

In [9]:
submission_queries = {}
print(f"{'Func':<5} | {'Optimizing...'}")
print("-" * 30)

for func_id in range(1, 9):
    # Ensure you have updated data to 17 points (10+7) before running
    X_known, y_known = load_data(func_id)
    next_x = suggest_next_point_hybrid(func_id, X_known, y_known)
    submission_queries[func_id] = next_x

print("\n" + "="*30)
print("FORMATTED SUBMISSION OUTPUT")
print("="*30)

for func_id, x_val in submission_queries.items():
    formatted_str = "-".join([f"{val:.6f}" for val in x_val])
    print(f"function_number: {func_id}: {formatted_str}")


FORMATTED SUBMISSION OUTPUT
function_number: 1: 0.931024-0.717283
function_number: 2: 0.511417-0.727013
function_number: 3: 0.309052-0.477563-0.466159
function_number: 4: 0.413347-0.464374-0.376904-0.384008
function_number: 5: 1.000000-1.000000-0.800000-1.000000
function_number: 6: 0.387153-0.355004-0.525003-0.783371-0.145920
function_number: 7: 0.000000-0.356489-0.455144-0.242287-0.255521-0.756803
function_number: 8: 0.000000-0.000000-0.187751-0.163048-0.674043-0.602089-0.000000-0.746610


What was sent

```
function_number: 1: 0.888647-0.933000
function_number: 2: 0.511417-0.727013
function number: 3: 0.554344-0.479092-0.490840
function_number: 4: 0.486697-0.324952-0.448014-0.369212
function_number: 5: 0.901030-1.000000-1.000000-1.000000
function_number: 6: 0.341863-0.412545-0.596785-0.795638-0.000000
function_number: 7: 0.000000-0.391975-0.550956-0.330957-0.483694-0.851496
function_number: 8: 0.000000-0.384962-0.000000-0.013207-0.274043-0.239238-0.000000-0.746610
```